In [2]:
import re
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer


from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

stop_words = set(stopwords.words('english'))

Using TensorFlow backend.


In [3]:
def preprocess(file):
    data = open(file, 'r',encoding="utf8")
    tweets = []

    for row_high in data:
        row = str(row_high.lower())
        row = row.split('\t')
        
        tweet = {}
        tweet['ID'] = row[0]
        tweet['sentiment'] = row[1]
        
        s = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', row[2]) 
        s=re.sub(r'-', ' ',s)
        s=re.sub(r'()[a-z0-9]*@[a-z0-9]*','',s)
        s=re.sub(r'[^a-z0-9 ]', '', s)
        s=re.sub(r'\b[0-9]+\b\s*','',s)
        content=re.sub(r'\b[a-z]\b','',s)
    
        words = content.split()
        words = [word for word in words if word not in stop_words] #removes stop words
        content = ' '.join([lemmatizer.lemmatize(word) for word in words]) #reduces every word to its stem
        tweet['text'] = content
        tweets.append(tweet)
        
    return tweets


In [ ]:
"""
Max entropy classifier
"""

target=[]
for i in range(len(sent_train)):
    if sent_train[i]=='positive':
        target.append(2)
    if sent_train[i]=='negative':
        target.append(0)
    if sent_train[i]=='neutral':
        target.append(1) 

test_target=[]
for i in range(len(sent_test3)):
    if sent_test3[i]=='positive':
        test_target.append(2)
    if sent_test3[i]=='negative':
        test_target.append(0)
    if sent_test3[i]=='neutral':
        test_target.append(1)


lm = LogisticRegression(solver='lbfgs',multi_class='auto',max_iter=500)

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(training)
X_test_counts = count_vect.transform(testing3)


tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
Test_X_Tfidf = tfidf_transformer.fit_transform(X_test_counts)


model = lm.fit(X_train_tfidf,target)
prediction = model.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(accuracy_score(prediction, test_target)*100)


sentiment=[]
for i in range(len(prediction)):
    if prediction[i]==0:
        sentiment.append('negative')
    if prediction[i]==1:
        sentiment.append('neutral')
    if prediction[i]==2:
        sentiment.append('positive')
        
results = dict(zip(Id_test,sentiment))
print(results)

In [4]:
"""
LSTM classifier
"""

data = sent_train
values = array(data)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
# binary encode
onehot_encoder = OneHotEncoder(categories='auto',sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)


data = sent_test
values = array(data)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
# binary encode
onehot_encoder = OneHotEncoder(categories='auto',sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
test_onehot_encoded = onehot_encoder.fit_transform(integer_encoded)


data = sent_dev
values = array(data)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
# binary encode
onehot_encoder = OneHotEncoder(categories='auto',sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
dev_onehot_encoded = onehot_encoder.fit_transform(integer_encoded)



tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(training)

X_train = tokenizer.texts_to_sequences(training)
X_test = tokenizer.texts_to_sequences(testing)
X_dev = tokenizer.texts_to_sequences(deving)


vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
X_dev = pad_sequences(X_dev, padding='post', maxlen=maxlen)


embeddings_dictionary = dict()
glove_file = open('glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()


embedding_matrix = np.zeros([vocab_size, 100])
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
        

NameError: name 'sent_train' is not defined

In [ ]:
model = Sequential()
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)
model.add(LSTM(32))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])


print(model.summary())

history = model.fit(X_train, onehot_encoded, batch_size=256, epochs=5, verbose=1, validation_data=(X_dev,dev_onehot_encoded))


score = model.evaluate(X_test, test_onehot_encoded, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

In [ ]:
data = sent_test3
values = array(data)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
# binary encode
onehot_encoder = OneHotEncoder(categories='auto',sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
test_onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

X_test = tokenizer.texts_to_sequences(testing3)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


score = model.evaluate(X_test, test_onehot_encoded, verbose=1)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

data = model.predict_classes(X_test, verbose=1)
sentiment=[]
for i in range(len(data)):
    if data[i]==0:
        sentiment.append('negative')
    if data[i]==1:
        sentiment.append('neutral')
    if data[i]==2:
        sentiment.append('positive')

results = dict(zip(Id_test,sentiment))